In [ ]:
#hide
#default_exp tools.query
from nbdev.showdoc import show_doc

# Query

> Shows results for queried experiments.

In [ ]:
#export
import warnings
warnings.filterwarnings('ignore')

import argparse
import sys
sys.path.append('.')
from collections import namedtuple
from IPython.display import display
# hpsearch api
import hpsearch.utils.experiment_utils as ut
import pdb


def query (pv = {}, pf = {}, pall=[], pexact=False, root= 'sac', 
           metric = 'cost_test', experiments=None, runs = None, op = 'min', stats=['mean'], 
           results=0, other_parameters=False):
    
    
    result_query = ut.query(folder_experiments=root, suffix_results='_'+metric, experiments = experiments,
                        classes=runs, parameters_fixed=pf, parameters_variable=pv, parameters_all = pall, exact_match=pexact, 
                        ascending=op=='min', stats=stats, min_results=results, query_other_parameters=other_parameters)
    
    if not other_parameters:
        result_query = result_query[1]
        result_query = result_query['stats']

    return result_query 

def do_query_and_show (pall=[], best=None, compact=0, exact=False, experiments=None, pf={}, last=None, 
                       metric='cost_test', op='min', other_parameters=False, input_range=None, results=0, 
                       root='sac', round=2, runs=None, show=False, stats=['mean'], pv={},
                       sort=None):
    
    df = query (pv = pv, pf = pf, pall=pall, pexact=exact, root= root, 
               metric = metric, experiments=experiments, runs = runs, op = op, stats=stats, 
               results=results, other_parameters=other_parameters)
    df = ut.replace_with_default_values (df)
    if sort is not None:
        assert sort in df.columns, f'sort must be a column in dataframe ({df.columns})'
        df = df.sort_values(by=sort, ascending=(op=='min'))
    if experiments is None:
        experiments = []
    if last is not None:
        experiments += range(df.index.max()-last+1, df.index.max()+1)
    if best is not None:
        experiments += list(df.index[:best])
    if input_range is not None:
        assert len(input_range) == 2
        experiments += range(input_range[0], input_range[1])
    if len(experiments) > 0: 
        df = df.loc[[x for x in df.index if x in experiments]]
    
    if (round is not None) and (round != 0):
        df[stats] = df[stats].round(round)
    display (df)
    
    print ('index: ')
    print (df.index)
    print ('min: {}, max: {}'.format(df.index.min(), df.index.max()))

    print ('with unique parameters:')
    _, df2 = ut.get_parameters_unique(df)
    if compact > 0:
        prev_cols = df2.columns.copy()
        df2, dict_rename = ut.compact_parameters (df2, compact)
        for k, kor in zip(df2.columns, prev_cols):
            print ('{} => {}'.format(k, kor))
    display (df2)
            
    if show:
        import hpsearch.visualization.plot_visdom as pv
        pv.plot_multiple_histories(df.index, root_folder=root,metrics=metric, parameters=None)
    

def main():
    default_root = 'sac'
    default_metric = 'cost_test'
    default_always = ''
    default_op = 'min' 

    #python print_table2.py -d 2 -f "dict(optim='sgd')"

    parser = argparse.ArgumentParser(description='show metrics in visdom browser')
    # Datasets
    parser.add_argument('-m','--metric', type=str, default=default_metric, help="metrics scores")
    parser.add_argument('--stats', type=str, nargs='+', default=['mean'],  help="statistics for multiple runs")
    parser.add_argument('--experiments', type=int, nargs='+', default=None,  help="experiment numbers")
    parser.add_argument('-r','--root', type=str, default=default_root)
    parser.add_argument('-v', type=str, default='{}', help='variable parameters')
    parser.add_argument('-f', type=str, default='{}', help='fixed parameters')
    parser.add_argument('-a', type=str, default='[]', help='all parameters')
    parser.add_argument('-e', '--exact', action= "store_true", help='exact match') 
    parser.add_argument('--last', type=int, default=None, help='include these last experiments') 
    parser.add_argument('--best', type=int, default=None, help='include these best experiments')
    parser.add_argument('--range', type=int, nargs='+', default=None, help='include this range of experiments')
    parser.add_argument('-c', '--compact', type=int, default=0, help='compact parameters to this number of characters') 
    parser.add_argument('--results', type=int, default=0, help='min number of results to consider') 
    parser.add_argument('-d', type=int, default=0)
    parser.add_argument('-s', '--show', action= "store_true")
    parser.add_argument('--other', action= "store_true")
    parser.add_argument('--always', type=str, default = default_always)
    parser.add_argument('--op', default=default_op, type=str)
    parser.add_argument('--round', default=2, type=int, help='round scores to this number of digits')
    parser.add_argument('--runs', default=None, type=int, nargs='+', help='query restricted to run number provided')
    parser.add_argument('--sort', default=None, type=str)
    pars = parser.parse_args()

    pars.v = eval(pars.v)
    pars.f = eval(pars.f)
    pars.a = eval(pars.a)
    pars.always = eval('dict(%s)' %pars.always)
    pars.f.update(pars.always)

    print ('dictionary of query terms=%s' %pars.f)

    if pars.d == 1:
        pars.root = 'squeezenet2'
    elif pars.d == 2:
        pars.root = 'squeezenet2msmt'
    elif pars.d == 3:
        pars.root = 'allexperiments'
    
    #pdb.set_trace()
    do_query_and_show (pall=pars.a, best = pars.best, compact = pars.compact, exact=pars.exact, experiments=pars.experiments,
                       pf = pars.f, last=pars.last, metric = pars.metric, op = pars.op, other_parameters=pars.other,
                       input_range=pars.range, results=pars.results, root= pars.root, round=pars.round, runs = pars.runs,
                       show=pars.show, stats=pars.stats, pv = pars.v, sort=pars.sort)
                    

## Test

### Produce data

In [ ]:
#collapse_output
from hpsearch.examples.dummy_experiment_manager import remove_previous_experiments, run_multiple_experiments

remove_previous_experiments()
run_multiple_experiments(nruns=5, noise=0.1)

experiment script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line: 125
processing hyper-parameter 0 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
returning registered experiment manager
returning experiment manager <hpsearch.examples.dummy_experiment_manager.DummyExperimentManager object at 0x7f3e4ec20550>
running experiment 0
run number: 0

parameters:
	epochs=5,
	offset=0.1,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 125
time spent on this experiment: 0.00022172927856445312
0 - validation_accuracy: 0.21389833508917952
0 - test_accuracy: 0.3564398908724477
finished experiment 0
processing hyper-parameter 0 out of 9
doing run 1 out of 5
fixed rate, multiple epochs values
running experiment 0
run number: 1

parameters:
	epochs=5,
	offset=0.1,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 125
time spent on this experiment: 0.0002706

current path: /mnt/athena/hpsearch/nbs/tools
current path: /mnt/athena/hpsearch/nbs/tools
current path: /mnt/athena/hpsearch/nbs/tools
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools



parameters:
	epochs=5,
	offset=0.1,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 125
time spent on this experiment: 0.0007255077362060547
4 - validation_accuracy: 0.4250798676823809
4 - test_accuracy: 0.3210649179845191
finished experiment 0
processing hyper-parameter 1 out of 9
doing run 0 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 0

parameters:
	epochs=5,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 125
time spent on this experiment: 0.00029349327087402344
0 - validation_accuracy: 0.6413611207342278
0 - test_accuracy: 0.43543364943740936
finished experiment 1
processing hyper-parameter 1 out of 9
doing run 1 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 1

parameters:
	epochs=5,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 125


fitting model with 5 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
noise: 0.1
current path: /mnt/athena/hpsearch/nbs/tools
fitting model with 5 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
noise: 0.1


2 - validation_accuracy: 0.42922981727441917
2 - test_accuracy: 0.5978008724456518
finished experiment 1
processing hyper-parameter 1 out of 9
doing run 3 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 3

parameters:
	epochs=5,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 125
time spent on this experiment: 0.00027108192443847656
3 - validation_accuracy: 0.34591117524367165
3 - test_accuracy: 0.5432926811184073
finished experiment 1
processing hyper-parameter 1 out of 9
doing run 4 out of 5
fixed rate, multiple epochs values
running experiment 1
run number: 4

parameters:
	epochs=5,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 125
time spent on this experiment: 0.0002543926239013672
4 - validation_accuracy: 0.2936839701044057
4 - test_accuracy: 0.5340896234969111
finished experiment 1
processing hyper-parameter 2 out of 9

...

fitting model with 30 epochs
epoch 0: accuracy: 0.13
epoch 1: accuracy: 0.16
epoch 2: accuracy: 0.19
epoch 3: accuracy: 0.22
epoch 4: accuracy: 0.25
epoch 5: accuracy: 0.28
epoch 6: accuracy: 0.31000000000000005
epoch 7: accuracy: 0.3400000000000001
epoch 8: accuracy: 0.3700000000000001
epoch 9: accuracy: 0.40000000000000013
epoch 10: accuracy: 0.43000000000000016
epoch 11: accuracy: 0.4600000000000002
epoch 12: accuracy: 0.4900000000000002
epoch 13: accuracy: 0.5200000000000002
epoch 14: accuracy: 0.5500000000000003
epoch 15: accuracy: 0.5800000000000003
epoch 16: accuracy: 0.6100000000000003
epoch 17: accuracy: 0.6400000000000003
epoch 18: accuracy: 0.6700000000000004
epoch 19: accuracy: 0.7000000000000004
epoch 20: accuracy: 0.6700000000000004
epoch 21: accuracy: 0.6400000000000003
epoch 22: accuracy: 0.6100000000000003
epoch 23: accuracy: 0.5800000000000003
epoch 24: accuracy: 0.5500000000000003
epoch 25: accuracy: 0.5200000000000002
epoch 26: accuracy: 0.4900000000000002
epoch 27:

running experiment 7
run number: 2

parameters:
	epochs=30,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 125
time spent on this experiment: 0.0009410381317138672
2 - validation_accuracy: 0.5871152434659008
2 - test_accuracy: 0.5114535480297215
finished experiment 7
processing hyper-parameter 7 out of 9
doing run 3 out of 5
fixed rate, multiple epochs values
running experiment 7
run number: 3

parameters:
	epochs=30,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 125
time spent on this experiment: 0.0008413791656494141
3 - validation_accuracy: 0.5967398965302781
3 - test_accuracy: 0.4571692286211587
finished experiment 7
processing hyper-parameter 7 out of 9
doing run 4 out of 5
fixed rate, multiple epochs values
running experiment 7
run number: 4

parameters:
	epochs=30,
	offset=0.3,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_expe

fitting model with 30 epochs
epoch 0: accuracy: 0.32999999999999996
epoch 1: accuracy: 0.36
epoch 2: accuracy: 0.39
epoch 3: accuracy: 0.42000000000000004
epoch 4: accuracy: 0.45000000000000007
epoch 5: accuracy: 0.4800000000000001
epoch 6: accuracy: 0.5100000000000001
epoch 7: accuracy: 0.5400000000000001
epoch 8: accuracy: 0.5700000000000002
epoch 9: accuracy: 0.6000000000000002
epoch 10: accuracy: 0.6300000000000002
epoch 11: accuracy: 0.6600000000000003
epoch 12: accuracy: 0.6900000000000003
epoch 13: accuracy: 0.7200000000000003
epoch 14: accuracy: 0.7500000000000003
epoch 15: accuracy: 0.7800000000000004
epoch 16: accuracy: 0.8100000000000004
epoch 17: accuracy: 0.8400000000000004
epoch 18: accuracy: 0.8700000000000004
epoch 19: accuracy: 0.9000000000000005
epoch 20: accuracy: 0.8700000000000004
epoch 21: accuracy: 0.8400000000000004
epoch 22: accuracy: 0.8100000000000004
epoch 23: accuracy: 0.7800000000000004
epoch 24: accuracy: 0.7500000000000003
epoch 25: accuracy: 0.720000000

running experiment 8
run number: 1

parameters:
	epochs=30,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 125
time spent on this experiment: 0.0010631084442138672
1 - validation_accuracy: 0.8720822162097538
1 - test_accuracy: 0.8760040387543436
finished experiment 8
processing hyper-parameter 8 out of 9
doing run 2 out of 5
fixed rate, multiple epochs values
running experiment 8
run number: 2

parameters:
	epochs=30,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_experiment_manager.py, line number: 125
time spent on this experiment: 0.0008668899536132812
2 - validation_accuracy: 0.8327177012547603
2 - test_accuracy: 0.7551789085593473
finished experiment 8
processing hyper-parameter 8 out of 9
doing run 3 out of 5
fixed rate, multiple epochs values
running experiment 8
run number: 3

parameters:
	epochs=30,
	offset=0.6,
	rate=0.03

script: /mnt/athena/hpsearch/hpsearch/examples/dummy_expe

fitting model with 30 epochs
epoch 0: accuracy: 0.63
epoch 1: accuracy: 0.66
epoch 2: accuracy: 0.6900000000000001
epoch 3: accuracy: 0.7200000000000001
epoch 4: accuracy: 0.7500000000000001
epoch 5: accuracy: 0.7800000000000001
epoch 6: accuracy: 0.8100000000000002
epoch 7: accuracy: 0.8400000000000002
epoch 8: accuracy: 0.8700000000000002
epoch 9: accuracy: 0.9000000000000002
epoch 10: accuracy: 0.9300000000000003
epoch 11: accuracy: 0.9600000000000003
epoch 12: accuracy: 0.9900000000000003
epoch 13: accuracy: 1.0200000000000002
epoch 14: accuracy: 1.0500000000000003
epoch 15: accuracy: 1.0800000000000003
epoch 16: accuracy: 1.1100000000000003
epoch 17: accuracy: 1.1400000000000003
epoch 18: accuracy: 1.1700000000000004
epoch 19: accuracy: 1.2000000000000004
epoch 20: accuracy: 1.1700000000000004
epoch 21: accuracy: 1.1400000000000003
epoch 22: accuracy: 1.1100000000000003
epoch 23: accuracy: 1.0800000000000003
epoch 24: accuracy: 1.0500000000000003
epoch 25: accuracy: 1.020000000000

### Run simple query

In [ ]:
do_query_and_show (metric='validation_accuracy', op='max')

total data examined: 9 experiments with at least 5 class_ids each


,epochs,offset,rate,num_results,mean
5,15.0,0.6,0.03,5.0,0.99
8,30.0,0.6,0.03,5.0,0.91
2,5.0,0.6,0.03,5.0,0.83
4,15.0,0.3,0.03,5.0,0.77
7,30.0,0.3,0.03,5.0,0.60
3,15.0,0.1,0.03,5.0,0.55
1,5.0,0.3,0.03,5.0,0.41
6,30.0,0.1,0.03,5.0,0.39
0,5.0,0.1,0.03,5.0,0.28


index: 
Int64Index([5, 8, 2, 4, 7, 3, 1, 6, 0], dtype='int64')
min: 0, max: 8
with unique parameters:


,epochs,offset,mean
5,15.0,0.6,0.99
8,30.0,0.6,0.91
2,5.0,0.6,0.83
4,15.0,0.3,0.77
7,30.0,0.3,0.60
3,15.0,0.1,0.55
1,5.0,0.3,0.41
6,30.0,0.1,0.39
0,5.0,0.1,0.28


### Run query that sorts by maximum

In [ ]:
do_query_and_show (metric='validation_accuracy', op='max', sort='max', stats=['mean', 'min', 'max'])

total data examined: 9 experiments with at least 5 class_ids each


,epochs,offset,rate,num_results,mean,min,max
5,15.0,0.6,0.03,5.0,0.99,0.96,1.00
8,30.0,0.6,0.03,5.0,0.91,0.83,1.00
4,15.0,0.3,0.03,5.0,0.77,0.62,0.90
2,5.0,0.6,0.03,5.0,0.83,0.76,0.89
7,30.0,0.3,0.03,5.0,0.60,0.51,0.74
6,30.0,0.1,0.03,5.0,0.39,0.19,0.66
1,5.0,0.3,0.03,5.0,0.41,0.29,0.64
3,15.0,0.1,0.03,5.0,0.55,0.49,0.59
0,5.0,0.1,0.03,5.0,0.28,0.18,0.43


index: 
Int64Index([5, 8, 4, 2, 7, 6, 1, 3, 0], dtype='int64')
min: 0, max: 8
with unique parameters:


,epochs,offset,mean,min,max
5,15.0,0.6,0.99,0.96,1.00
8,30.0,0.6,0.91,0.83,1.00
4,15.0,0.3,0.77,0.62,0.90
2,5.0,0.6,0.83,0.76,0.89
7,30.0,0.3,0.60,0.51,0.74
6,30.0,0.1,0.39,0.19,0.66
1,5.0,0.3,0.41,0.29,0.64
3,15.0,0.1,0.55,0.49,0.59
0,5.0,0.1,0.28,0.18,0.43


In [ ]:
#hide
from nbdev.export import *
notebook2script(recursive=True)

Converted hpconfig.ipynb.
Converted manager_factory.ipynb.
Converted dummy_experiment_manager.ipynb.
Converted example_experiment.ipynb.
Converted example_experiment_manager.ipynb.
Converted experiment_manager.ipynb.
Converted index.ipynb.
Converted change_manager.ipynb.
Converted query.ipynb.
Converted experiment_utils.ipynb.
Converted organize_experiments.ipynb.
Converted metric_correlation.ipynb.
Converted plotly.ipynb.
